In [25]:
# 인사이트 경제부분 스크래핑 1~6페이지까지 

from bs4 import BeautifulSoup
import requests
import json
import csv

insight_news_info = []
cnt = 0 # 뉴스 number


# csv변환시키는 함수
def convert_to_csv(csv_name, fieldnames, infor):
    try:
        with open(f'./${csv_name}.csv', 'w', newline='') as c:
            fieldnames = fieldnames
            writer = csv.DictWriter(c, fieldnames=fieldnames) 
            writer.writeheader()

            for news in infor:
                print(news)
                writer.writerow(news) 
    except:
        pass

    
# Dict형태로 변환 함수
def dict_infor(**kwargs):
#     news_info = []
    context = {}
    for k,v in kwargs.items():
        context[k] = v
    news_info.append(context)
#     print(news_info)
    return news_info



for num in range(1,6):
    response = requests.get(f'https://www.insight.co.kr/section/money?page={num}')
    soup = BeautifulSoup(response.text, 'html.parser')

    # 메인 페이지 기사 ul
    ul = soup.select_one('body > div.content > div > div.section-wrap > div.section-list > ul')
    # 메인 페이지 기사리스트들 li
    lis = ul.select('li')
    # 현재 카테고리 네임
    category = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > span')
    press = soup.select_one('body > div.content > div > div.section-wrap > div.section-gnb > a').text
    # 기사 리스트 for문
    # 프리뷰 기사 img, title, date가져옴
    # 현재 카테고리 각각의 리스트에 함께 나오게 하기 위해 단순히 위에서 받은 new_category밑에다 써줌
    
    for li in lis:
        cnt += 1
        preview_img = li.select_one('li > div > a.section-list-article-image')
        preview_img = preview_img['style'].split("(")[1].split(')')[0]
        title = li.select_one('li > div > a.section-list-article-title').text.strip()
        news_category = category.text.split('>')[1].strip()
        date = li.select_one('li > div > span.section-list-article-byline')
        link = li.select_one('li > div > a.section-list-article-title')['href']
        code = link.split('/')[4]
        
        # code로 내용을 가지고 오는대신 다시한번 requests와 parser이용
        news_url = requests.get(link)
        news_url_html = BeautifulSoup(news_url.text, 'html.parser')
        
        # 세부 컨텐츠 안에 주요 내용 selector css         
        content = news_url_html.select_one('body > div.content > div > div.news-container > div.news-wrap > div.news-article > div.news-article-memo')
        # 세부 컨텐츠 안에 이미지, 이미지 여러개이므로 list에 담아줌
        news_image = content.select('img')
        image_url_list = []
        for image in news_image:
            image_url_list.append(image['src'])
        
        # 기자 이름이 날짜와 함께 나오는 경우가 있음 "." 이 있는 경우는 기자 네임이 있는 경우라 .을 기준으로 나누어서 날짜만 가져옴
        if '·' in date.text:
            date = date.text.strip().split('·')[1]

        else:
            date = date.text.strip()
        
        # 이하 똑같은 형식
        insight_news_info = dict_infor(num=cnt, press=press,news_code=code, news_category=news_category, date=date, preview_img=preview_img, title=title, content=content.text,img=image_url_list,ref=link )

# print(insight_news_info)
convert_to_csv('insight_news_infor',['num', 'press', 'news_code', 'news_category', 'date','preview_img','title', 'content', 'img', 'ref' ],insight_news_info)




{'num': 1, 'press': '인사이트', 'news_code': '341484', 'news_category': '생활경제', 'date': '2021-06-02 17:53:23', 'preview_img': 'https://img.insight.co.kr/static/2021/06/02/300/img_20210602174733_zw6y3wil.jpg', 'title': '"주유소 가기 무서울 정도"···물가 역대급으로 상승해 살기 더 힘들어졌다', 'content': '\n뉴시스[뉴시스] 이승재 기자 = 기름값이 고공 행진을 이어가는 가운데 파, 마늘, 소·돼지고기 등 장바구니 물가까지 뛰면서 최근 2개월 연속 2%대 물가 상승이 이어졌다. 당분간 이 흐름이 지속될 것으로 예상되면서 인플레이션을 우려하는 목소리도 나온다.정부는 연간 물가 안정 목표치인 2%를 넘기지는 않을 것으로 보고 있다. 최근의 상승세는 지난해 저물가에 따른 기저 요인이 크게 작용했고, 하반기부터는 농축수산물, 석유류 등 물가 여건이 개선될 것이라는 이유에서다.2일 통계청 자료를 보면 지난 4월과 5월 소비자물가지수는 지난해 같은 기간과 비교해 각각 2.3%, 2.6% 뛰었다.이처럼 2개월 연속 2%대 오름세가 이어진 것은 지난 2018년 11월(2.0%) 이후 2년 6개월 만이다. 당시 소비자물가지수는 9월부터 3개월 동안 2%대 상승률을 유지했다.최근 물가 흐름은 석유류(23.3%)와 농축수산물(4.0%) 가격 상승에 따른 것으로 보인다.뉴시스실제로 이 두 품목의 지난달 물가 기여도는 1.8%포인트(p)에 달한다. 전체 물가 상승률(2.6%)에서 70%가량을 차지했다는 뜻이다. 지난 4월의 경우 이 수치는 1.5%p로 65%의 기여도를 보였다.특히, 석유류는 지난해 5월 저점(-18.7%)을 찍었던 기저효과가 그대로 반영됐다.같은 기간 두바이유 가격은 배럴당 30.5달러에서 66.3달러로 2배 넘게 뛰었고, 휘발유 가격은 리터당 1255원에서 1541원으로 올랐다.농축수산물의

# 6/27

### - 장고 db에 넣어보기
### - content html형태 그대로 스크래핑 후 장고 db에 넣어보기
### - 두가지 전부 화면에 내용 나오는지 확인하기